In [17]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W



In [2]:
spark = SparkSession.builder.getOrCreate()

22/07/10 12:11:35 WARN Utils: Your hostname, francesco-TravelMate-P238-M resolves to a loopback address: 127.0.1.1; using 192.168.1.129 instead (on interface wlp3s0)
22/07/10 12:11:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/10 12:11:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [40]:
df = spark.read.format('csv').option('header', 'true').load("owid-covid-data.csv")
df.select("iso_code", 'location', 'population', 'total_cases').show(3)


+--------+-----------+----------+-----------+
|iso_code|   location|population|total_cases|
+--------+-----------+----------+-----------+
|     AFG|Afghanistan|38928341.0|        1.0|
|     AFG|Afghanistan|38928341.0|        1.0|
|     AFG|Afghanistan|38928341.0|        1.0|
+--------+-----------+----------+-----------+
only showing top 3 rows



In [51]:
# df.schema.json()
# 15 стран с наибольшим процентом переболевших на 31 марта
res_df = df.select("iso_code", 'location', 'population', 'total_cases', 'date')\
.filter(F.col("date") <= '2022-03-31' ).groupBy('iso_code').agg({'population': 'max', 'total_cases' : 'sum'})
res_df = res_df.withColumn('percent', F.col("sum(total_cases)") / F.col("max(population)") )
res_df.sort('percent', ascending = False).show(15)

+--------+----------------+---------------+------------------+
|iso_code|sum(total_cases)|max(population)|           percent|
+--------+----------------+---------------+------------------+
|     AND|       1819252.0|        77265.0|23.545615738044393|
|     SMR|        638957.0|        33938.0|18.827184866521304|
|     MNE|     1.0873903E7|       628062.0|17.313422878633002|
|     CZE|    1.69000815E8|     10708982.0|15.781221314967192|
|     QAT|     4.4008361E7|      2881060.0|15.275058832513034|
|     BHR|     2.5859756E7|      1701583.0|15.197469650319732|
|     LUX|       9300282.0|       625976.0|14.857250118215395|
|     USA|   4.677925907E9|    331002647.0|14.132593649621176|
|     ISR|    1.18035557E8|      8655541.0|13.636993574405112|
|     PAN|     5.7985556E7|      4314768.0|13.438858358085533|
|     SVN|     2.5270665E7|      2078932.0|12.155599605951517|
|     BEL|    1.36567367E8|     11589616.0|11.783597230486325|
|     ARM|     3.3388579E7|      2963234.0|11.267614707

In [50]:
# 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
res_df = df.select("iso_code", 'new_cases', 'date')\
.filter(F.col("date") <= '2021-03-31' )\
    .filter(F.col("date") >= '2021-03-28' )\
        .filter(~F.col('iso_code').like('OWID_%') )\
        .groupBy('iso_code').agg({'new_cases' : 'sum'})
# res_df = res_df.withColumn('percent', F.col("sum(total_cases)") / F.col("max(population)") )
res_df.sort('sum(new_cases)', ascending = False).show(10)

+--------+--------------+
|iso_code|sum(new_cases)|
+--------+--------------+
|     BRA|      258385.0|
|     IND|      250041.0|
|     USA|      240814.0|
|     TUR|      138067.0|
|     FRA|      135904.0|
|     POL|       99992.0|
|     ITA|       72446.0|
|     DEU|       60719.0|
|     ARG|       47432.0|
|     UKR|       42502.0|
+--------+--------------+
only showing top 10 rows



22/07/10 17:28:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 11928523 ms exceeds timeout 120000 ms
22/07/10 17:28:14 WARN SparkContext: Killing executors is not supported by current scheduler.


In [64]:
res_rus = df.filter(F.col('iso_code') == 'RUS').filter(F.col("date") <= '2021-03-31' )\
    .filter(F.col("date") >= '2021-03-28' ).select("iso_code", 'new_cases', 'date')
w = W.orderBy('date')
res_rus = res_rus.withColumn('yesterday_cases', F.lag('new_cases', 1).over(w))
res_rus.withColumn('diff_cases', F.col('new_cases') - F.col('yesterday_cases') ).show()


22/07/10 23:04:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+---------+----------+------+----------+
|iso_code|new_cases|      date|    tt|diff_cases|
+--------+---------+----------+------+----------+
|     RUS|   8979.0|2021-03-28|  null|      null|
|     RUS|   8589.0|2021-03-29|8979.0|    -390.0|
|     RUS|   8162.0|2021-03-30|8589.0|    -427.0|
|     RUS|   8156.0|2021-03-31|8162.0|      -6.0|
+--------+---------+----------+------+----------+



In [57]:
w